<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S4_lambdaswitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Negative feedback loop. The Lambda Phage Switch

## **Introducción**
*Nota: Es conveniente que amplíes este resumen teórico con el capítulo 5 de An Introduction to Systems Biology de Uri Alon.*

Los sistemas biológicos pueden decidir entre diferentes estados, haciéndolos permanentes. El bacteriófago lambda, un virus que infecta bacterias *E. coli*, es un buen ejemplo de este proceso; cuando el fago infecta una célula, debe “elegir” entre la **vía lítica**, en la que se reproduce rápidamente y destruye la célula huésped, o la **vía lisogénica**, en la que se integra en el genoma bacteriano y permanece latente.

Esta elección está controlada por un circuito molecular centrado en dos proteínas clave: **CI** (el represor λ) y **Cro**, que se reprimen mutuamente. De este modo tenemos un **interruptor biológico**, que establece un estado: niveles altos de CI mantienen el estado lisogénico, mientras que niveles altos de Cro desencadenan la vía lítica.

Lo que hace que este sistema sea especialmente interesante es su **naturaleza biestable**. Igual que un interruptor de luz que solo puede estar “encendido” o “apagado”, el interruptor lambda tiende a estabilizarse en uno de dos estados posibles, sin posiciones intermedias. Esta biestabilidad surge de la represión mutua entre CI y Cro, combinada con la **unión cooperativa** (las proteínas se unen como dímeros) y con tasas cuidadosamente ajustadas de producción y degradación.

En esta simulación podrás explorar cómo funciona este interruptor, modelando las interacciones moleculares clave mediante un sistema de ODEs. El modelo incorpora las características:

*   Unión cooperativa de proteínas (representada por coeficientes de Hill).
*   Represión mutua mediante unión a operadores.
*   Producción basal (“expresión con fuga”).
*   Degradación proteica.

Al ajustar parámetros como las tasas de producción (β), las tasas de degradación (α), las constantes de unión (K) y las condiciones iniciales, podremos investigar:

*   Cómo pequeñas diferencias en los niveles iniciales de proteínas conducen a resultados radicalmente distintos.
*   Las condiciones necesarias para mantener la bistabilidad.
*   El papel de la cooperatividad en la creación de un comportamiento tipo interruptor.
*   Cómo la degradación proteica afecta la estabilidad de cada estado.




## Banco de módulos

In [1]:
#@title Módulo 0. Librerías
# -*- coding: utf-8 -*-
# =========================
# Módulo 0: Librerías y estilo
# =========================
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp

plt.style.use('classic')
sns.set_theme(style="whitegrid")

# Paleta consistente
PALETTE = {
    "CI": "#1f77b4",   # azul
    "Cro": "#d62728",  # rojo
    "nullcline_CI": "#1f77b4",
    "nullcline_Cro": "#d62728",
    "vector_field": "#7f7f7f"
}


In [3]:
#@title Módulo 1. Modelo Lambda Switch
# =========================
# Módulo 1: Modelo del Lambda Switch
# =========================
def hill_repression(x, K, n):
    """
    Función de represión tipo Hill: g(x) = 1 / (1 + (x/K)^n)
    x: concentración del represor
    K: constante de semisaturación
    n: coeficiente de Hill (cooperatividad)
    """
    return 1.0 / (1.0 + (x / K)**n)

def model_lambda_switch(t, y, params):
    """
    Modelo mínimo del interruptor lambda con represión mutua y expresión basal.

    Variables:
      y = [CI, Cro]

    Ecuaciones:
      dCI/dt  = beta_CI * g_Cro(Cro) + beta0_CI - alpha_CI * CI
      dCro/dt = beta_Cro * g_CI(CI)  + beta0_Cro - alpha_Cro * Cro

    Donde g_X(.) es la función de represión Hill del represor correspondiente.

    params esperados:
      params["CI"]  = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      params["Cro"] = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      (opcional) params["input"] = función u(t) para modular producción (ver abajo)
      (opcional) params["input_effect"] = dict con claves "CI" y/o "Cro" y ganancia k

    Nota: Si se desea entrada externa, se implementa como modulación multiplicativa
          de la producción máxima: beta_eff = beta * (1 + k * u(t)).
    """
    CI, Cro = y

    # Parám. CI
    beta_CI  = params["CI"]["beta"]
    beta0_CI = params["CI"]["beta0"]
    alpha_CI = params["CI"]["alpha"]
    K_CI     = params["CI"]["K"]
    n_CI     = params["CI"].get("n", 2)

    # Parám. Cro
    beta_Cro  = params["Cro"]["beta"]
    beta0_Cro = params["Cro"]["beta0"]
    alpha_Cro = params["Cro"]["alpha"]
    K_Cro     = params["Cro"]["K"]
    n_Cro     = params["Cro"].get("n", 2)

    # Entrada externa opcional (p. ej., señal de daño SOS)
    u = 0.0
    if "input" in params and callable(params["input"]):
        u = params["input"](t)  # Sintaxis correcta: función de entrada evaluada en t

    # Modulación opcional de betas por entrada externa
    if "input_effect" in params:
        k_CI  = params["input_effect"].get("CI", 0.0)
        k_Cro = params["input_effect"].get("Cro", 0.0)
        beta_CI_eff  = beta_CI  * (1.0 + k_CI  * u)
        beta_Cro_eff = beta_Cro * (1.0 + k_Cro * u)
    else:
        beta_CI_eff, beta_Cro_eff = beta_CI, beta_Cro

    # Represiones Hill
    g_Cro_on_CI  = hill_repression(Cro, K_Cro, n_Cro)  # Cro reprime CI
    g_CI_on_Cro  = hill_repression(CI,  K_CI,  n_CI)   # CI  reprime Cro

    # Ecuaciones dinámicas
    dCI_dt  = beta_CI_eff  * g_Cro_on_CI + beta0_CI  - alpha_CI  * CI
    dCro_dt = beta_Cro_eff * g_CI_on_Cro + beta0_Cro - alpha_Cro * Cro

    return [dCI_dt, dCro_dt]
